In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import theano.tensor as tt
import seaborn as sns


#print('Running on PyMC3 v{}'.format(pm.__version__))
# If you get an error like: non-constant-expression cannot be narrowed 
# then comment these lines out
# If you know how to actually fix it, please let me know!
#import theano
#theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"


# Counting Your Customers the Hard Way
## A Tutorial
by Anthony Alford, [genesys.com](https://www.genesys.com/)

## Table of Contents

### [Introduction](#introduction)

### [Generative Model](#generative-model)

### [PyMC3 Implementation](#pymc3-implementation)

### [Prediction](#prediction)



<a id="introduction"></a>
## Introduction

This notebook demonstrates [Ascarza and Hardie's model](https://www0.gsb.columbia.edu/.../4587/ascarza_hardie_churn.pdf) for customer usage and churn. The model consists of a latent *commitment* process, an observed *usage* process that is observed every time period, and *renewal* process that is observed every *n* periods.

### Setup
I've included a Docker file, two shell scripts, and a pip requirements file. The Docker file is based on the Jupyter Docker Stacks `scipy-notebook` image.

If you'd like to run this notebook in Docker:
 * Open a terminal and `cd` to the directory where you cloned this repo
 * Execute the `build-docker.sh` shell script. This builds an image and tags it `churn-and-usage`. The image will have the packages listed in the [scipy-notebook documentation](https://jupyter-docker-stacks.readthedocs.io/en/latest/using/selecting.html#jupyter-scipy-notebook_) plus any listed in `requirements.txt`
 * Execute the `run-docker.sh` shell script. This wil start up a Docker container that is running Jupyter. It will also map the working directory (i.e., this repo) to a directory called `work`, where this notebook can be found.

<a id="generative-model"></a>
## Generative Model

First we will code a function to model this process and generate some test data. Then we we will use PyMC3 to estimate the parameters of a model from data (both real and simulated).

Notation (from Ascarza and Hardie, but with zero-based indexing):
* $i$ denotes each customer: $i = {1, 2, \ldots, I}$
* $t$ denotes the usage time unit (periods). For each $i$, total of $T_i$ observations (note: for our purposes, all customers have the same number of observations)
* $n$ is number of usage periods associated with a contract period


At each time $t$, each customer $i$ has a latent commitment state $S_{it}$, which follows a Markov process, with $K$ states ${0, 1, \ldots, K-1}$, with $0$ corresponding to the lowest level of commitment and $K-1$ to the highest. A customer's states change over time according to a transition matrix $\Pi =\{\pi_{jk}\}$, with $j, k \in {1, \dots, K}$. This process also models the renewal behavior: if a customer's latent state is 0 at a renewal period, the customer will churn.

The probability that customer $i$ is in commitment state $k$ at beginning of its lifetime is determined by the vector $Q = \{q_1 ,\ldots, q_K \}$

For a customer in in state $k$, the usage process generates an observed value (e.g., number of purchase) in period $t$. The observation is drawn from a Poisson distribution with a parameter $\lambda_{it}$ that is composed of a state dependent parameter $\theta_k$ that varies depending on the underlying level of commitment (which varies over time) and an individual specific parameter $\alpha_i$ that remains constant over time. Thus: $ \lambda_{it}~\big|~[S_{it}=k]~=~\alpha_i\theta_k $

The vector $\boldsymbol\theta = \{\theta_k\}, k = 1, \ldots, K$ of state-specific parameters allows the each customer’s mean usage levels to change over time. In order to satisfy the condition $$0 < \theta_1<\theta_2<\ldots<\theta_K $$ the vector components are reparameterized as $$ \theta_{\tau}~=~\theta_{\tau-1} + e^{\gamma_\tau}, ~~\forall~\tau > 1$$


## Simulated Data - Parameters
To simulate data, we need the following parameters:
* Q - vector of the initial state probabilities ($Q$)
* PI - transition matrix for the Markov process ($\Pi$)
* THETA - vector of state-dependentent usage parameters ($\boldsymbol\theta$), composed of
 * th0 - state zero usage parameter ($\theta_0$)
 * g1, g2, etc - higher-state usage parameters ($\gamma_{\tau}$)
* A - vector customer specific usage parameters ($\alpha_i$)

In [ ]:
Q_actual=np.array([0.0, 0.50, 0.50])

PI_actual=np.array([[0.85, 0.15, 0.0],
       [0.2, 0.55, 0.25],
       [0.0, 0.45, 0.55]])


th0_actual=5.0
G_actual = np.array([2.5, 3.0])


A_actual=np.array([5.0, 10.0, 7.0, 20.0])


## Simulated Data - Generation Function
This function implements the model described above. It outputs three numpy arrays:
* The (unobserved) states for each customer 
* The churn events (if any) for each customer
* The observed usage for each customer

Each array's shape is $(I,T)$. That is: one row for each customer and one column for each time period observed. We are using a common observation window for all customers.

Besides the model parameters listed above, the function also has a flag to "force" the generation of non-churning customer data. In other words, we can use it to simulate data where no customers churn. This was handy for simulating a set of customers with a common observation window.

The function can also be given an initial set of states for each customer. This might be handy later.

In [ ]:
def generate_data(Q, PI, th0, G, ALPHA, num_samples, renewal_period, force_no_churn, initial_state=None):

    states = []
    samples = []
    events = []

    # construct Theta vector from parameters
    THETA = np.zeros(Q.shape[0])
    THETA[0] = th0
    for i in range(1,Q.shape[0]):
        THETA[i] = THETA[i-1] + np.exp(G[i-1])
    
    for i in range(0,ALPHA.shape[0]):
        if initial_state is None:
            state = np.random.choice(Q.shape[0], 1, p=Q)[0]
        else:
            state = np.random.choice(Q.shape[0],1,p=PI[initial_state[i]])[0]     

        churned = False
        for j in range(0,num_samples):  

            # if the force_no_churn flag is set, and churn condition met
            # the choose a new state
            if churned == False and force_no_churn == True and state == 0 and (j+1) % renewal_period == 0:
                while state == 0:
                    state = np.random.choice(Q.shape[0],1,p=PI[state])[0]

            # save current state
            states = np.append(states, state)

            # usage calculation in current state
            if churned == False:
                lam = ALPHA[i] * THETA[state]
                sample = np.random.poisson(lam=lam)
            else:
                sample = 0

            # churn event in current state
            if churned == False:
                if state == 0 and (j+1) % renewal_period == 0:
                    churned = True
                    event = 1
                else:
                    event = 0
            else:
                event = 0

            samples = np.append(samples, sample)
            events = np.append(events, event)

            # choose next state
            if churned == False:
                state = np.random.choice(Q.shape[0],1,p=PI[state])[0]
            else:
                state = 0

    states = np.reshape(states, (ALPHA.shape[0], num_samples))
    events = np.reshape(events, (ALPHA.shape[0], num_samples))
    samples = np.reshape(samples, (ALPHA.shape[0], num_samples))
    return [states, events, samples]

Now let's get some simulated data

In [ ]:
total_observations = 12
renewal_period = 3

observed_usage = np.array(generate_data(Q_actual, PI_actual, th0_actual, G_actual, A_actual, 
                                     total_observations, renewal_period, True)[2])

<a id="pymc3-implementation"></a>
## PyMC3 Implementation

To implement the model in PyMC3, we need two custom distributions, for the commitment process and the usage process respectively. We'll create Python classes to represent these, passing the appropriate values to the constructor, and implementing the `logp` method to calculate the log-likelihood.

NOTE: in developing this part of the code, I found [Helmut Strey's Hidden Markov Model repo](https://github.com/hstrey/Hidden-Markov-Models-pymc3) invaluable.

### Committment Process
The commitment process also includes the renewal or process:
>We use the categorical distribution...to draw the augmented states, and control for the path restrictions (due to the contractual specification) by truncating the categorical distribution function in the renewal periods (t = 1, n + 1, 2n + 1, ..).

In other words, if a customer has not been observed to churn in a past renewal period, the customer *cannot* have been in state 0. This implies that `logp` should be -Inf, so we use the `bound` function. To make that logic work with linear algebra, we pass in a "mask" vector, which is length $T-1$. It represents the time periods (starting from the second, or $t=1$, period) where renewal is possible: 1 at renewal periods and 0 everywhere else. We know a valid customer state history cannot be in state $s=0$ at those points. Therefore, if we take the dot product of a customers vector of state history with this mask must always be 0 for a valid state history.

In [ ]:
from pymc3.distributions.dist_math import bound

class CommitmentProcess(pm.Categorical):
    """
    
    """

    def __init__(self, PI=None, Q=None, renewal_mask=None, num_states=3, 
                 *args, **kwargs):
        super(pm.Categorical, self).__init__(*args, **kwargs)
        self.PI = PI
        self.Q = Q
        self.renewal_mask = renewal_mask
        self.k = num_states
        self.mode = tt.cast(0,dtype='int64')
    
    def logp(self, x):
        
        log_p = 0.
        for i in range(0, self.shape[0]):
            p_it = self.PI[x[i,][:-1]]
            x_t = x[i,][1:]
            x_0 = tt.stack([x[i,][0]])
            mask = self.renewal_mask
        
            log_p_i = pm.Categorical.dist(self.Q).logp(x_0) + tt.sum(pm.Categorical.dist(p_it).logp(x_t))
        
            # Restrction: if not churned, cannot be in state 0 at a renewal period
            log_p = log_p + bound(log_p_i, tt.dot(mask, x_t)>0)

        return log_p


### Usage Process

Customer's usage likelihood:

$$
\begin{align*}
L(\boldsymbol{\theta}, \alpha_i~\big|~\tilde{S_i}=\tilde{s_i},data) &= \prod_{t=1}^{T_i} P(Y_{it}=y_{it}~\big|~S_{it}=k,\boldsymbol{\theta},\alpha_i) \\
&= \prod_{t=1}^{T_i} \frac{e^{-\alpha_i\theta_i}(\alpha_i\theta_k)^{y_{it}} }{y_{it}!} \\
&= \prod_{t=1}^{T_i} \frac{e^{-\lambda_{it}}(\lambda_{it})^{y_{it}} }{y_{it}!} \\
\end{align*}
$$


Which is just the Poisson likelihood, with a different lambda for each observation.

In [ ]:
class UsageProcess(pm.Discrete):
    """
    
    """

    def __init__(self, alpha=None, th0=None, G=None, states=None, num_states=3,
                 *args, **kwargs):
        super(UsageProcess, self).__init__(*args, **kwargs)
        self.alpha = alpha
        self.th0 = th0
        self.G = G
        self.states = states
        self.num_states = num_states
        self.mean = 0.

        
    def logp(self, x):
        states = self.states

        # build theta vector from the components
        theta = tt.concatenate([tt.stack([self.th0]), tt.exp(self.G)])
        zero = tt.zeros_like(theta)
        
        for i in range(1, num_states):
            theta = theta + tt.concatenate([zero[-i:], theta[0:self.num_states-i]])
        
        # build lambda matrix: theta is row vector, alpha is column
        # labmda is the outer-product of the two
        Lambda = tt.outer(self.alpha,theta)
        
        log_p = 0.
        for i in range(0, self.shape[0]):
            lam_it = Lambda[i][states[i,]]
            y_it = x[i]
            log_p_i = tt.sum(pm.Poisson.dist(mu=lam_it).logp(y_it))
            
            log_p = log_p + log_p_i

        return log_p


### PyMC3 Model
We are now ready to use PyMC3 to estimate the model parameters. First, we'll setup the priors. From the paper:
* $Q$ and $\Pi$: Dirichlet prior with equal probabilities
* $\alpha_i$: gamma distribution with shape and scale parameter r
* $r$: diffuse gamma distribution prior (shape and scale parameters being 0.01)
* $\theta_0$: uniform prior over the interval (0,10) 
* $\gamma_\tau$: diffuse normal priors (with mean 0 and variance 10000)



In [ ]:
# Sampling params
chains = 1
draws = 3000
post_burn_in = 1000

# Model params
num_states = 3
num_custs = observed_usage.shape[0]
obs_len = observed_usage.shape[1]
renewal_mask = np.where(((np.arange(1,obs_len)+1) % renewal_period)!=0,0,1)

# This lets us avoid trying out a lot of invalid state possibilities
states_test_val = np.ones((num_custs, obs_len))

from scipy import optimize
with pm.Model() as model:
    Q = pm.Dirichlet('Q', a=np.ones((num_states)) + 1., shape=(num_states))    
    PI = pm.Dirichlet('PI', a=np.ones((num_states,num_states)) + 1., shape=(num_states,num_states))    
    r = pm.Gamma('r', alpha=0.01, beta=0.01)
    A = pm.Gamma('A',alpha=r, beta=r, shape=((num_custs)))    
    th0 = pm.Uniform('th0',lower=0.0,upper=10.0)
    G = pm.Normal('G',mu=np.zeros(num_states-1), sd=np.ones(num_states-1)*10000., shape=(num_states-1))

    states = CommitmentProcess('states', PI=PI, Q=Q, renewal_mask = renewal_mask, num_states=num_states, shape=(num_custs,obs_len), testval=states_test_val)
    usage = UsageProcess('usage', alpha=A, th0=th0, G=G, states=states, num_states=num_states, shape=(num_custs), observed=observed_usage)
    
    start = pm.find_MAP(method='Powell')
    step1 = pm.Metropolis(vars=[r,PI,Q,A,G,th0,usage])
    step2 = pm.CategoricalGibbsMetropolis(vars=[states])
    trace = pm.sample(draws, start=start, step=[step1,step2], chains=1)


### Save the Trace
We can save the trace to a file for later use.

In [ ]:
pm.backends.ndarray.save_trace(trace, directory='./traces/trace1', overwrite=True)


## Model Convergence
From the paper
>We burn-in the first 95,000 iterations and use the following 5,000 iterations to draw from the posterior marginal distributions. Convergence of the parameters was diagnosed visually, looking at the historical draws of all parameters, and also using the Gelman-Rubin statistic R, which compares the ratio of the pooled chain variance with the within chain variance, for a model estimated using multiple and disperse starting values.

We can use the trace plot to visually diagnose convergence.

In [ ]:
_ = pm.traceplot(trace[-post_burn_in:])

<a id="prediction"></a>
## Prediction
Now that we've estimated the parameters of the model, we can use them to predict future customer behavior.

### Point Estimate
One source of estimates is to take the average of the trace values (ignoring some initial section of the trace, as "burn-in").

In [ ]:
Q_avg = np.average(trace['Q'][-post_burn_in:],axis=0)
PI_avg = np.average(trace['PI'][-post_burn_in:],axis=0)
G_avg = np.average(trace['G'][-post_burn_in:],axis=0)
th0_avg = np.average(trace['th0'][-post_burn_in:],axis=0)
A_avg = np.average(trace['A'][-post_burn_in:],axis=0)
last_observed_state = np.floor(np.average(trace['states'][-post_burn_in:],axis=0)[:,-1] + 0.5).astype(int)

Now we can take these paramater estimates and give them to the data generator to predict future states and usage. This is where the `initial_state` param comes in: we can use an estimate of the last state each customer was in.

In [ ]:
prediction_periods = 3
prediction = generate_data(Q_avg, PI_avg, th0_avg, G_avg, A_avg, prediction_periods, renewal_period, False, last_observed_state)
np.reshape(prediction, (3, A_avg.shape[0], prediction_periods))

### Sample from the Trace
Instead of taking the average of the trace, we can iterate over the trace data and generate a prediction for each of the parameter values. This will give us a result that shows the uncertainty of the prediction.

In [ ]:
states = []
events = []
usage = []

prediction_periods = 3
for i in range(post_burn_in):
    Q_sample = trace['Q'][i]
    PI_sample = trace['PI'][i]
    G_sample = trace['G'][i]
    th0_sample = trace['th0'][i]
    A_sample = trace['A'][i]
    last_observed_state = trace['states'][i][:,-1]
    st, ev, us = generate_data(Q_sample, PI_sample, th0_sample, G_sample, A_sample, prediction_periods, renewal_period, False, last_observed_state )
    states.append(st)
    events.append(ev)
    usage.append(us)
    
states = np.reshape(states, (post_burn_in, A_sample.shape[0], prediction_periods))
events = np.reshape(events, (post_burn_in, A_sample.shape[0], prediction_periods))
usage = np.reshape(usage, (post_burn_in, A_sample.shape[0], prediction_periods))

This gives us a lot of possible outcomes. We can do things like plot the distribution of the usage for one customer in a certain period

In [ ]:
_ = sns.distplot(usage[:,0,0])

or we can look at the probability of the customers churning

In [ ]:
np.average(events, axis=0)